In [1]:
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta
import classes_func
from classes_func import *

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Creating a search instance with Search('City',start_day,end_day)
Barcelona = Search('Valencia','2024-02-25','2024-03-01')

In [3]:
#Remove Cookies
#Find destination button
Barcelona.browser.find_element(by='xpath',value= '//button[@id="onetrust-reject-all-handler"]').click()

In [4]:
#Remove Google Sign in
# Define your wait timeout (adjust as needed)
wait_timeout = 10
wait = WebDriverWait(Barcelona.browser, wait_timeout)

try:
    # Switch to the iframe by index (if needed, adjust the index)
    Barcelona.browser.switch_to.frame(0)  # You may need to adjust the index based on your HTML structure

    # Now you can interact with elements within the iframe
    # Wait for the "Not Now" button to be clickable and click it
    not_now_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='TvD9Pc-Bz112c ZYIfFd-aGxpHf-FnSee' and @aria-label='Cerrar']")))
    not_now_button.click()

except Exception as e:
    print(f"An error occurred while handling the Google Sign-In pop-up: {e}")

finally:
    # Switch back to the default content
    Barcelona.browser.switch_to.default_content()


In [5]:
#Inputing city
Barcelona.input_city()

In [6]:
Barcelona.open_date_box()

In [7]:
#Select Date_start
Barcelona.date_selector(Barcelona.start_day)

In [8]:
#Select Date_end
Barcelona.date_selector(Barcelona.end_day)

In [9]:
#Search for Results
Barcelona.search_results()

In [10]:
time.sleep(5)
# Removing the Genius pop-up only requires a clicking outside the pop-up
obscured = check_obscures(Barcelona.browser,'//div[@class="efdb2b543b e4b7a69a57"]',"xpath")
try:
    if not obscured:
        Barcelona.browser.find_element(by='xpath',value= '//div[@class = "abcc616ec7 cc1b961f14 c180176d40 f11eccb5e8 ff74db973c"]//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e f4552b6561"]').click()
    else:
        print('no element blocking the path')
except Exception as e:
    print(f"An error occurred: {e}")
#html.b_firefox.b_firefox_121.supports_inline-block.supports_flexbox_unprefixed.supports_fontface.supports_hyphens.hasJS body#b2searchresultsPage.bookings2.b2.searchresults.es.lang_is_ltr.header_reshuffle.no_bg_img.nobg.user_center.app_user_center.sb_gradient_border.b-sprite-3.ds-traveller-header.lx_cwv_font_swap.bigblue_std_sm.bigblue_std_lg.system-font.iq-x-bar.iq-x-bar-new div.b9720ed41e.cdf0a9297c div.eb33ef7c47 div.f0c216ee26.c676dd76fe.b5018b639f div.c0528ecc22 div.dd5dccd82f div.ffd93a9ecb.dc19f70f85.eb67815534 div.abcc616ec7.cc1b961f14.c180176d40.f11eccb5e8.ff74db973c button.a83ed08757.c21c56c305.f38b6daa18.d691166b09.ab98298258.deab83296e.f4552b6561

Message: Element <div class="efdb2b543b e4b7a69a57"> is not clickable at point (779,247) because another element <div class="f7c2c6294c"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:333:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31



In [11]:
#Total number of Result pages
pages = Barcelona.result_pages()
pages

28

In [12]:
#Barcelona.scrape_results(pages)
# Also if it's going to slow, adjust the waiting times in the class_function
Barcelona.scrape_results(pages)#Add the number of pages you'll like to scrape

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27


In [13]:
Barcelona.df.shape

(512, 8)

In [14]:
import concurrent.futures
from tqdm import tqdm

descriptions = []
linkey = Barcelona.df['Link']

def scrape_description(link):
    URL = f'{link}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Getting Data
    div2 = soup.find('div', {'id': 'hp_hotel_name'})
    div = soup.find('div', {'id': 'property_description_content'})
    
    # Find all p elements with a specific class within the div
    if div is not None:
        specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
        if div2 is not None:
            # Find all h2 elements with a specific class within div2
            specific_class_h_elements = div2.find_all('h2', class_='d2fee87262 pp-header__title')
        else:
            specific_class_h_elements = []
    
        descriptions = []
        for h, p in zip(specific_class_h_elements, specific_class_p_elements):
            descriptions.append((h.get_text(strip=True), p.get_text(strip=True)))
    else:
        # If 'div' is not found, return an empty list
        descriptions = []
    
    return descriptions

threads = os.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers=threads-2) as executor:
    results = list(tqdm(executor.map(scrape_description, linkey), total=len(linkey), desc="Processing Links"))

# Flatten the results list
description_result = [item for sublist in results for item in sublist]
description_df = pd.DataFrame(description_result, columns=['Hotels', 'Descriptions'])


Processing Links: 100%|████████████████████████████████████████████████████████████████████████████| 512/512 [05:36<00:00,  1.52it/s]


In [15]:
description_df.shape

(512, 2)

In [16]:
# Apply left join between main df and desscriptions
Barcelona.df = pd.merge(Barcelona.df, description_df, how='left', on='Hotels')

In [17]:
Barcelona.df = Barcelona.df.drop_duplicates()
Barcelona.df[0:50]

,Hotels,Distance,District,Price,Rating,Stars,Comments,Link,Descriptions
0,Valencia Luxury - Sea Port II,"a 2,4 km del centro","Camins al Grau, Valencia",€ 796,"7,8",3 de 5,686 comentarios,https://www.booking.com/hotel/es/sea-port-ii.e...,Valencia Luxury - Sea Port II se encuentra en ...
1,Melia Valencia,"a 3,5 km del centro","Benicalap, Valencia",€ 1.409,"8,5",4 de 5,3.437 comentarios,https://www.booking.com/hotel/es/melia-valenci...,Este Meliá es un hotel moderno situado junto a...
3,Hermoso apartamento,"a 3,4 km del centro","Quatre Carreres, Valencia",€ 518,"8,6",3 de 5,35 comentarios,https://www.booking.com/hotel/es/hermoso-apart...,"Hermoso apartamento se encuentra en Valencia, ..."
4,Hi Sweet room,a 3 km del centro,"Rascanya, Valencia",€ 252,"8,4",3 de 5,382 comentarios,https://www.booking.com/hotel/es/aladin-piso.e...,"Hi Sweet room se encuentra en Valencia, a 3,4 ..."
5,Zerka Centro,a 250 m del centro,"Ciutat Vella, Valencia",€ 329,"7,7",3 de 5,1.977 comentarios,https://www.booking.com/hotel/es/rumbau-center...,Zerka Centro ofrece alojamiento con aire acond...
6,B&B El Mercat,a 500 m del centro,"Ciutat Vella, Valencia",€ 210,"6,2",2 de 5,632 comentarios,https://www.booking.com/hotel/es/b-amp-b-el-me...,El B&B El Mercat se encuentra en el distrito C...
7,Rotaret,a 150 m del centro,"Ciutat Vella, Valencia",€ 245,"7,8",3 de 5,2.340 comentarios,https://www.booking.com/hotel/es/rotaret.es.ht...,Rotaret está muy bien situado en el barrio de ...
8,City Center Guest house,"a 1,6 km del centro","La Saïdia, Valencia",€ 273,"6,7",3 de 5,700 comentarios,https://www.booking.com/hotel/es/the-towers-of...,City Center Guest house se encuentra en Valenc...
9,Balcón Al Mar,"a 3,8 km del centro","Poblados marítimos, Valencia",€ 384,"8,3",4 de 5,686 comentarios,https://www.booking.com/hotel/es/balcon-al-mar...,El Balcón al Mar está situado junto al puerto ...
10,Malvarrosa Beach Rooms,"a 4,6 km del centro","Poblados marítimos, Valencia",€ 286,"5,4",4 de 5,1.665 comentarios,https://www.booking.com/hotel/es/chicote.es.ht...,El Malvarrosa Beach Rooms se encuentra a solo ...


In [18]:
#define downloads path
downloads_path = "./downloads_datasets/"
valencia_25_01_data = Barcelona.df

valencia_25_01_data.to_csv(os.path.join(downloads_path, "valencia_25_01_data.csv"), index=False)

In [19]:
Barcelona.browser.quit()